# Experimental Code

<i>Version 1.0</i>

## Dependencies

The package, [bitsets](https://bitsets.readthedocs.io/en/stable/), provides a memory-efficient pure-python immutable ordered set data type for working with large numbers of subsets from a predetermined pool of objects.

In [1]:
from bitsets import bitset

Algebra's are defined in JSON format.

In [2]:
import json

## Allen's Interval Algebra using Bitsets

In [9]:
class Algebra(object):

    def __init__(self, filename):
        """An algebra is created from a JSON file containing the algebra's
        relation and transitivity table definitions.
        """
        with open(filename, 'r') as f:
            self.algebra_dict = json.load(f)

        self.name = self.algebra_dict["Name"]
        self.description = self.algebra_dict["Description"]
        self.rel_info = self.algebra_dict["Relations"]
        self.fullrelset = bitset(self.name, tuple(self.rel_info.keys()))
        self.identity = self.fullrelset.supremum

        # Setup the transitivity table used by Relation Set multiplication
        self.transitivity_table = dict()
        tabledefs = self.algebra_dict["TransTable"]
        for rel1 in tabledefs:
            self.transitivity_table[rel1] = dict()
            for rel2 in tabledefs[rel1]:
                self.transitivity_table[rel1][rel2] = self.fullrelset(tuple(tabledefs[rel1][rel2]))
                
    def __str__(self):
        return f"<{self.name}: {self.description}>"
    
    def converse(self, rel_or_relset):
        '''Return the converse of a relation (str) or relation set (bitset).'''
        if isinstance(rel_or_relset, str):
            return self.rel_info[rel_or_relset]["Converse"]
        else:
            return self.fullrelset((self.converse(r) for r in rel_or_relset.members()))
        
    def relname(self, rel):
        return self.rel_info[rel]["Name"]
    def reltran(self, rel):
        return self.rel_info[rel]["Transitive"]

In [10]:
allen = Algebra("IntervalAlgebra.json")
print(allen)

<LinearTimeIntervalAlgebra: Allen's algebra of proper time intervals>


In [11]:
allen.converse(allen.fullrelset(('B','M','OI')))

LinearTimeIntervalAlgebra(['BI', 'MI', 'O'])

In [12]:
allen.converse('B')

'BI'

In [13]:
allen.identity

LinearTimeIntervalAlgebra(['B', 'BI', 'D', 'DI', 'E', 'F', 'FI', 'M', 'MI', 'O', 'OI', 'S', 'SI'])

In [14]:
allen.relname('B')

'Before'

In [15]:
#allen.transitivity_table